In [1]:
#Setup
#Shows each individual plot along with which model best fits it
import pickle, math
import pandas as pd
import sqlite3
import random
import os, re
exec(open("./Imports.py").read())
#execfile('Imports.py')
# _add_modules_path()
import Modules.Funcs as funcs
from Modules.Classes import Simulation
from Modules.Classes import CopyTweak
from Modules.Classes import CopyTweakRep
from Modules.Classes import Packer
from Modules.Classes import PackerRep
from Modules.Classes import ConjugateJK13
from Modules.Classes import RepresentJK13
from Modules.Classes import NegatedSpace
from Modules.Classes import NConjugateJK13
from Modules.Classes import NRepresentJK13
from Modules.Classes import NPacker
from Modules.Classes import NCopyTweak

from scipy.stats import stats as ss
from scipy.stats import binom_test
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style("whitegrid")


In [2]:
# Extract the pickled output that is produced from
# individual_model_gridsearch_CHTC.py and saves the best params as a pickle

appendkey = ['finalparmsll','chunkstartparms','parmnames']
appendOnce = ['parmnames'] #append this key only once per participant
removekey = ['bestparmsll','chunkidx','startparms']

data = dict()
datasetsAll = []
modelsAll = []
print_ct = 0

dataset = "corner"
if not dataset in datasetsAll:
    datasetsAll += [dataset]

joefiles = ["joe_ind_gs_best_params_corner_chunk0.p",
            "joe_ind_gs_best_params_corner_chunk1.p",
            "joe_ind_gs_best_params_corner_chunk2.p"]
for fn in joefiles:
    with open(os.path.join("pickles/newpickles/",fn), "rb" ) as f:
        datachunk = pickle.load(f)
    models = datachunk.keys()
    #Remove fit_weights from models, since it's not a model
    fit_weights = False
    if not dataset in data:
        data[dataset] = dict()            
    for model in models:
        #Skip fit_weights
        if model=="fit_weights":
            continue
        if not model in modelsAll:
            modelsAll += [model]
        #Cycle through each participant and remove noninformative keys
        #print(model)
        nppt = len(datachunk[model])
        ppts = datachunk[model].keys()
        for ppt in ppts:
            #Remove noninformative keys
            for key in removekey:
                datachunk[model][ppt].pop(key,None)
            if not model in data[dataset]:
                #If new model, copy over everythin
                data[dataset][model] = dict()
                data[dataset][model][ppt] = datachunk[model][ppt]
            else:
                #Otherwise, just append the stuff as ndicated by appendkey
                if not ppt in data[dataset][model]:
                    data[dataset][model][ppt] = dict()
                    for key in appendkey:
                        data[dataset][model][ppt][key] = datachunk[model][ppt][key]
                else:
                    for key in appendkey:
                        if not key in appendOnce:
                            data[dataset][model][ppt][key] = np.concatenate((data[dataset][model][ppt][key], datachunk[model][ppt][key]),0)
    




In [3]:
#Find best parms for each model in each dataset
pickledir = 'pickles/'
for dataset in datasetsAll:
    for model in modelsAll:
        if model in data[dataset].keys():
            ppts = data[dataset][model].keys()
            for ppt in ppts:
                results_all = data[dataset][model][ppt]['finalparmsll']
                ind = np.argsort(results_all[:,-2]) #-2 is the column containing of LL
                results_all_sorted = results_all[ind]
                results_best = results_all_sorted[0,:]
                #startp_sorted = data[dataset][model][ppt]['startparms'][ind]
                data[dataset][model][ppt]['finalparmsll'] = results_all_sorted
                #data[dataset][model][ppt]['startparms'] = startp_sorted
                data[dataset][model][ppt]['bestparmsll'] = results_best

    # save final result in pickle    
    dst = dataset
    results = data[dataset]
    results['fit_weights'] = fit_weights
    with open('{}joe_ind_gs_best_params_{}.p'.format(pickledir,dst),'wb') as f:
        #pass 
        pickle.dump(results, f)


In [4]:
# Specify default dataname
dbname = 'experiment-corner.db'#raw data
dataname_def = 'corner'#bestparms also comes from here

#plotting options
STAT_LIMS =  (-1.0, 1.0)
#Fit to only last trial?
fitlast = False
#Make plots?
doplots = True
saveplots = True
if fitlast:
    showlast = True
else:
    showlast = False #True = Show last even if fitting to all?
#Bootstrap parameters
nbootstraps = 1000
savedirsub = 'individual_modelcomp_{}'.format(dataname_def)
if fitlast and showlast:
    savedir = savedirsub + '/fitlast'
elif showlast:
    savedir = savedirsub + '/showonlylast/'
elif fitlast:
    savedir = savedirsub + '/fitlast'
else:
    savedir = savedirsub + '/showall'
    
if saveplots:
    if not os.path.isdir(savedirsub):
        os.mkdir(savedirsub)
    if not os.path.isdir(savedir):
        os.mkdir(savedir)
        

#Some plotting options
font = {'family' : 'DejaVu Sans',
        'weight' : 'regular',
        'size'   : 15}

#Specify simulation values
N_SAMPLES = 10000
WT_THETA = 1.5
MIN_LL = 1e-10


participant_def = 'all'
unique_trials_def = 'all'
dataname = dataname_def
exec(open("./validate_data.py").read())


bestparmdb = "pickles/joe_ind_gs_best_params_{}".format(src)

plt.rc('font', **font)

# get data from pickle
with open(pickledir+src, "rb" ) as f:
    trials = pickle.load( f, fix_imports=True, encoding="latin1")

# get best params pickle
#bestparmdb = "pickles/chtc_gs_best_params_all_data_e1_e2.p"
#bestparmdb = "pickles/chtc_gs_best_params_corrs.p"
with open(bestparmdb, "rb" ) as f:
    best_params_t = pickle.load( f )

#Rebuild it into a smaller dict
best_params = dict()
for pi in trials.participants:
    best_params[pi] = dict()
    for modelname in best_params_t.keys():
        if modelname=="fit_weights":
            continue
        best_params[pi][modelname] = dict()
        for i,parmname in enumerate(best_params_t[modelname][pi]['parmnames']):
            parmval = best_params_t[modelname][pi]['bestparmsll']
            best_params[pi][modelname][parmname] = parmval[i]
modelList = [Packer,RepresentJK13]#,NPacker,NRepresentJK13]
#modelList = [CopyTweak,CopyTweakRep,Packer, RepresentJK13,]                            

#Specify plot order
modelPlotOrder = np.array([[Packer,RepresentJK13],[NPacker,NRepresentJK13]])
#modelPlotOrder = np.array([[CopyTweak,CopyTweakRep],[Packer,RepresentJK13]])

#Prepare matched database    
matchdb='../cat-assign/data_utilities/cmp_midbot.db'
        
unique_trials = 'all'
trials.task = task

In [5]:
#Extract experiment data from db

con = sqlite3.connect(dbname)
participants = pd.read_sql_query("SELECT participant, condition from participants", con)
generation = pd.read_sql_query("SELECT * from generation", con)
alphas = pd.read_sql_query("SELECT * from alphas", con)
stimuli = pd.read_sql_query("SELECT * from stimuli", con).values
con.close()


In [6]:
#Go through each participant and get their fits for each model
ll = []
ll_trials = []
tally = np.zeros(len(modelList))
wins = []
pptcond_tally = dict()
nice_cond_tally = dict()
for ppt in trials.participants:#[318]:
    ppt = int(ppt)
    print('Ppt: ' + str(ppt))
    llrow = []
    if fitlast:
        extract_trials = 3
    else:
        extract_trials = 'all'
    conditionStr = participants.condition[participants.participant==ppt].values[0]
    if conditionStr not in pptcond_tally.keys():
        pptcond_tally[conditionStr] = 0
    pptcond_tally[conditionStr] += 1
    pptconstr = conditionStr[0:2]
    if "Corner" in conditionStr:
        pptconstr += conditionStr[-1] #Add info on whether square or circle
    
    trialppt = Simulation.extractPptData(trials, ppt=ppt, unique_trials=extract_trials)
    #Hacky code, but necessary for now
    #For categories to be ints
    for ti,trial in enumerate(trialppt.Set):
        trialppt.Set[ti]['categories'] = [np.array(tt,dtype=int) for tt in trial['categories']]
        trialppt.Set[ti]['response'] = [np.array(tt,dtype=int) for tt in trial['response']]


    #Get list of lls and winning model
    for model in modelList:
        params = best_params[ppt][model.model]
        ppt_trial_ll = trialppt.loglike(params=params,model=model,parmxform=False)
        llrow += [ppt_trial_ll]
    bestmodel = llrow.index(np.min(llrow))
    bestmodelstr = modelList[bestmodel].modelshort
    ll += [[ppt]+llrow]
    tally[bestmodel] += 1
    wins += [bestmodel]
    if not bestmodelstr in nice_cond_tally.keys():
        nice_cond_tally[bestmodelstr] = dict()
    if not conditionStr in nice_cond_tally[bestmodelstr].keys():
        nice_cond_tally[bestmodelstr][conditionStr] = 0        
    nice_cond_tally[bestmodelstr][conditionStr] += 1
    #Prepare stuff fot plotting
    if doplots:
        
        #Sort trial obj by trial number - really only applicable if onlylast is false                                                                                                                   
        trialOrder = []
        maxbeta = 0
        for t,trialobj in enumerate(trialppt.Set):
            #nbeta = len(trialobj['categories'][1])
            nbeta = 0
            for ii,new_exm in enumerate(trialobj['categories']):
                if not ii == 0: #Ignore alpha cat
                    nbeta += len(new_exm)
            trialppt.Set[t]['trial'] = nbeta
            trialOrder += [nbeta]
            maxbeta = max(nbeta,maxbeta)
        trialOrder = np.argsort(trialOrder)
        wrap_ax = trialppt.Set[0]['wrap_ax'][0]
        if wrap_ax==1:
            wrapstr = '<-Wrapped->'
        else:
            wrapstr = ''
        #Add dummy last trial to plot gradients for hypothetical fifth beta
        respSet = trialppt.Set[trialOrder[maxbeta]]['response']
        if hasattr(respSet[0],'__len__'):
            for resps in respSet:
                if len(resps)>0:
                    resp = resps
        else:
            resp = respSet
        newcatB = np.append(trialppt.Set[trialOrder[maxbeta]]['categories'][1],resp)
        newcatAll = [trialppt.Set[trialOrder[maxbeta]]['categories'][0],newcatB]
        trialppt.add(response=0,categories=newcatAll, participant = ppt,wrap_ax = wrap_ax)
        trialOrder = np.append(trialOrder,maxbeta+1)
        
        if showlast:
            ntrials = 1
            plottrials = [len(trialppt.Set)-1] #plot only last trial
        else:
            ntrials = len(trialppt.Set)
            plottrials = range(ntrials)
        f,ax = plt.subplots(ntrials,len(modelList),figsize = (8, 2*ntrials+2.5))
        for trial in plottrials:
            categories = [trialppt.stimuli[i,:] for i in trialppt.Set[trialOrder[trial]]['categories'] if len(i)>0]
            A = categories[0]
            respSet = trialppt.Set[trialOrder[trial]]['response']
            if hasattr(respSet[0],'__len__'):
                for ri,resps in enumerate(respSet):
                    if len(resps)>0:
                        respCat = ri
                        resp = trialppt.stimuli[resps,:]
            else:
                resp = trialppt.stimuli[respSet,:]
            if len(categories)>1:
                if respCat==1:
                    #include the response                                                                                                                  
                    B = np.append(categories[1],resp,axis=0)
                    if showlast:
                        #Include response in generatin of ps                
                        categories[1] = B
                elif respCat==2:
                    if len(categories)<3:
                        C = resp
                    else:
                        C = np.append(categories[2],resp,axis=0)
                    if showlast:
                        #Include response in generatin of ps                
                        categories[2] = C
            else:
                #categories += [resp]
                B = resp
                C = []
            ps = []
            ll_trial = []
            for m,model in enumerate(modelList):
                if m==bestmodel:
                    winstr = '*'
                else:
                    winstr = ''
                if ntrials>1:
                    ax[0,m].set_title('{}{}{}, ll={:.2f}'.format(winstr,model.modelshort,winstr,llrow[m]))
                else:                    
                    ax[m].set_title('{}{}{}, ll={:.2f}'.format(winstr,model.modelshort,winstr,llrow[m]))
                params = best_params[ppt][model.model]
                #Plot heatmap for each model
                ps += [model(categories,params,trialppt.stimrange,wrap_ax=wrap_ax).get_generation_ps(trialppt.stimuli,1,'generate')]
                #Get lls for each trial step
                ll_trial += [trialppt.loglike(params=params,model=model,parmxform=False,whole_array=True)]

            #Plot the individual plots
            plotct = 0
            plotVals = []
            psMax = 0
            psMin = 1
            #Get range                                                                                                                                     
            for ps_el in ps:
                psMax = max(psMax,ps_el.max())
                psMin = min(psMin,ps_el.min())

            #Normalise all values                                                                                                                          
            psRange = psMax-psMin
            for i,ps_el in enumerate(ps): #each ps element correspond to a model
                plotct += 1
                gps = funcs.gradientroll(ps_el,'roll')[:,:,0]
                ps_ElRange = gps.max()-gps.min();
                plotVals += [(gps-gps.min())/ps_ElRange]                                                                             
                betacol = ['green' for bi in range(len(B))]
#                 if task is 'generate':
#                     betacol[len(B)-1] = 'orange'
                if ntrials>1:
                    betacol[len(B)-1] = 'orange'
                    if trial == ntrials-1:
                        betacol[len(B)-1] = [0,0,0,0]
                    im = funcs.plotgradient(ax[trial,i], plotVals[i], A, B, clim = STAT_LIMS, cmap = 'PuOr',beta_col=betacol,gammas=C)
                    ax[trial,i].set_ylabel('Trial {}\n{}'.format(trial,wrapstr))
                    if trial<ntrials-1:
                        ax[trial,i].set_xlabel('ll_trial = {:.2f}'.format(ll_trial[i][trial]),fontsize=10)
                    #else:
                        
                else:
                    im = funcs.plotgradient(ax[i], plotVals[i], A, B, clim = STAT_LIMS, cmap = 'PuOr',beta_col=betacol,gammas=C)
#                     ax[i].set_ylabel('Trial {}'.format(trial))
        #Save fig
        if saveplots:
            savedirext = '{}/{}'.format(savedir,bestmodelstr)
            if os.path.isdir(savedirext):
                plt.savefig('{}/{}_{}_{}'.format(savedirext,pptconstr,int(ppt),bestmodelstr[0]))
            else:
                os.mkdir(savedirext)
                plt.savefig('{}/{}_{}_{}'.format(savedirext,pptconstr,int(ppt),bestmodelstr[0]))
        plt.close()
        ll_trials += [np.array(ll_trial).T]


Ppt: 9


/Users/sourguyxian/anaconda2/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:64: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.


Ppt: 10
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis rea

/Users/sourguyxian/anaconda2/envs/p37/lib/python3.7/site-packages/ipykernel_launcher.py:151: RuntimeWarning: invalid value encountered in true_divide



Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. D

...

Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up

Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up

Ppt: 102
Ppt: 103
Ppt: 104
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
M

Ppt: 107
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis reached. Up density ratio = 1.00000
Max wrapped axis reached. Down density ratio = 1.00000
Max wrapped axis re

Max wrapped axis reached. Up density ratio = 19.90676
Max wrapped axis reached. Down density ratio = 17.94841
Max wrapped axis reached. Up density ratio = 88.72848
Max wrapped axis reached. Down density ratio = 74.53179
Max wrapped axis reached. Up density ratio = 41.37974
Max wrapped axis reached. Down density ratio = 37.15386
Max wrapped axis reached. Up density ratio = 88.72087
Max wrapped axis reached. Down density ratio = 74.52569
Max wrapped axis reached. Up density ratio = 2.10473
Max wrapped axis reached. Down density ratio = 2.04472
Max wrapped axis reached. Up density ratio = 88.72850
Max wrapped axis reached. Down density ratio = 74.53177
Max wrapped axis reached. Up density ratio = 9.47198
Max wrapped axis reached. Down density ratio = 8.76260
Max wrapped axis reached. Up density ratio = 88.72848
Max wrapped axis reached. Down density ratio = 74.53179
Max wrapped axis reached. Up density ratio = 19.90676
Max wrapped axis reached. Down density ratio = 17.94841
Max wrapped ax

In [7]:
#Report number of model victories in nice form:
conditions = ['Corner_S','Corner_C']
#Print header
print("Condition   PACKER  Represent  Total    Binom_test")
for condition in conditions:
    packer_count = nice_cond_tally['PACKER'][condition]
    rep_count = nice_cond_tally['Representative'][condition]
    total_count = packer_count + rep_count
    bt = binom_test(packer_count,total_count)
    print("{}      {}        {}      {}      {:.5f}".format(condition,
                                                  packer_count,
                                                  rep_count,
                                                  total_count,
                                                       bt) )

#Print totals
bt = binom_test(tally[0],tally[0] + tally[1])
print("Total         {}        {}      {}      {:.5f}".format(int(tally[0]),
                                               int(tally[1]),
                                               int(tally[0] + tally[1]),
                                                 bt))



Condition   PACKER  Represent  Total    Binom_test
Corner_S      34        12      46      0.00164
Corner_C      23        20      43      0.76079
Total         57        32      89      0.01055


In [8]:
#Individual-level log likelihoods
ll

[[9, 15.524080296193674, 16.827946313390864],
 [10, 13.38531909397231, 8.841266145327388],
 [11, 11.58149172810051, 14.276705251339555],
 [12, 16.580512992986165, 14.66890778039533],
 [14, 15.084946847676427, 17.502632468502934],
 [15, 11.622513211425638, 17.502632468502934],
 [16, 8.536453253931773, 17.50263246850293],
 [17, 13.799182729213419, 12.60280027810508],
 [20, 3.401197381662156, 12.509428989548494],
 [21, 11.894511892855965, 12.492443074275798],
 [22, 11.361263328511201, 10.962466929856479],
 [24, 16.84796157119076, 17.50263246850293],
 [25, 8.219548407407753, 17.502632468502934],
 [26, 17.469666292563772, 16.793010956549686],
 [27, 12.940412226941593, 12.549364653800927],
 [29, 11.275380217594975, 17.502632468502934],
 [30, 17.397994403827674, 7.502636972893214],
 [32, 3.58351893845611, 8.11029965056177],
 [33, 11.107731682174707, 11.153464321005195],
 [37, 16.84796157119076, 17.245944457994852],
 [38, 16.70808723854864, 6.350835067443203],
 [39, 11.891449062060136, 15.4527